In [5]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show, curdoc, push_notebook
from bokeh.models import ColumnDataSource,HoverTool,ColorBar
from bokeh.plotting import figure, show, output_notebook
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bokeh.layouts import layout
from bokeh.layouts import row
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from bokeh.models import ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import ImageURL
from bokeh.io import curdoc, show
from bokeh.models.markers import Circle
from bokeh.models.widgets import Dropdown
from functools import partial
#from shapely.geometry import Point
#from shapely.ops import transform
#import pyproj
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.layouts import widgetbox,row,column
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool,HoverTool,LinearColorMapper,
     BasicTicker,
    PrintfTickFormatter,
    ColorBar,
     LogColorMapper,Slider, Button,LabelSet
)
from bokeh.models import (
    ColumnDataSource, HoverTool, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper,
)
from bokeh.palettes import Plasma256
import numpy as np

from bokeh.models import ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import ImageURL
from bokeh.io import curdoc, show
from bokeh.models.markers import Circle
from bokeh.models.widgets import Dropdown
from functools import partial
#from shapely.geometry import Point
#from shapely.ops import transform
#import pyproj
from bokeh.models.glyphs import Rect, Circle, Text

output_notebook()


building = np.loadtxt('BUILDING DATA.txt')
locations = pd.read_csv('RFID PATHWAY DATA v2.txt',delimiter='\t')
rfid_names = pd.read_csv('OCCUPANTS RFID ASSIGNMENTS.txt', delimiter = '\t', names=['Id','Person'])
locations = locations.join(rfid_names,on='Person',how='left', lsuffix='_id', rsuffix='_name')

x = []
y = []

for i in range(building.shape[0]):
    for j in range(building.shape[1]):
        if building[i][j]==1:
            x.append(j)
            y.append(building.shape[0]-i)
locations
print(max(x))
print(max(y))
print(min(x))
print(min(y))

Loading BokehJS ...

90
61
0
1


In [6]:
def graph(doc):
    def update(attr, new, old):
        temp = locations[(locations["Time"] == hour.value)]
        source.data = dict(x=temp["xcor"],y=temp["ycor"],ids=temp["Id"], person = temp['Person_name'])
    def animate_update():
        hr = hour.value + 1
        if hr > 837:
            hr = 0
        hour.value = hr
        
    def animate():
        global playing
        playing = True
        if button_start.label == '► Play':
            button_start.label = '❚❚ Pause'
            doc.add_periodic_callback(animate_update, 50)
        else:
            button_start.label = '► Play'
            doc.remove_periodic_callback(animate_update)
            
    def animate_stop():
        global playing
        if playing:
            doc.remove_periodic_callback(animate_update)
            hour.value = 1
            playing = False
    
    source0 = ColumnDataSource(data =dict(x=x,y=y))
    xdr = Range1d(start=0, end=91)
    ydr = Range1d(start=0, end=62)
    plot = Plot(title = None, x_range=xdr, y_range=ydr)
    sq = Rect(x="x",y="y", fill_alpha=1, width=1, height = 1)
    plot.add_glyph(source0,sq)
    #p = Plot(title = None, x_range=xdr, y_range=ydr)
    temp = locations[(locations["Time"] == 1)]
    
    source = ColumnDataSource(data =dict(x=temp["xcor"],y=temp["ycor"],ids=temp["Id"], person = temp['Person_name']))
    
    ids = [str(x) for x in range(82 + 1)]
    color_mapper = CategoricalColorMapper(palette=Plasma256, factors=ids)
    glyph2 = Circle(
        x="x",
        y="y",
        size=4,
        fill_color = "red",
        fill_alpha=0.8,
        line_color='#7c7e71',
        line_width=0.5,
        line_alpha=0.5,)
    plot.add_glyph(source, glyph2)
    glyph3 = Text(x='x', y='y', text='ids',x_offset=8, y_offset=8, text_font_size="10pt", text_color="blue", text_align='center')
    plot.add_glyph(source, glyph3)
    hour = Slider(start=1, end=837, value=1, step=1, title="Time")
    hour.on_change('value', update)
    button_start = Button(label='► Play', width=60)
    button_start.on_click(animate)
    button_stop = Button(label='► Stop', width=60)
    button_stop.on_click(animate_stop)
    controls = widgetbox([hour,button_start,button_stop], width=250)
    r1 = row([controls,plot])
    doc.add_root(r1)
    
    print(r1)

In [7]:
output_notebook()
app = Application(FunctionHandler(graph))
doc = app.create_document()
show(app, notebook_url='localhost:8888')

Loading BokehJS ...

Row(id='9b4359e0-dda0-4bcb-ad66-819d48345570', ...)


In [8]:
xdr = Range1d(start=0, end=91)
ydr = Range1d(start=0, end=62)
p = Plot(title = None, x_range=xdr, y_range=ydr)
temp = locations[(locations["Time"] == 1)]
#print(temp["xcor"])
source = ColumnDataSource(data =dict(x=temp["xcor"],y=temp["ycor"],ids=temp["Id"], person = temp['Person_name']))
    
ids = [str(x) for x in range(82 + 1)]
color_mapper = CategoricalColorMapper(palette=Plasma256, factors=ids)
glyph2 = Circle(
    x="x",
    y="y",
    size=6,
    fill_color={'field': 'ids', 'transform': color_mapper},
    fill_alpha=0.8,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,)
p.add_glyph(source, glyph2)
show(p, notebook_handle=True)